In [ ]:
import argparse
import cv2 as cv
import numpy as np
import xml.etree.ElementTree as ET
from lxml import etree
import os

In [ ]:
!pwd
%cd /content/drive/MyDrive/
!unzip card_data.zip

/content
/content/drive/MyDrive
Archive:  card_data.zip
   creating: card_data/
  inflating: card_data/10_template.jpg  
  inflating: card_data/11_template.jpg  
  inflating: card_data/14_template.jpg  
  inflating: card_data/15_template.jpg  
  inflating: card_data/18_template.jpg  
  inflating: card_data/19_template.jpg  
  inflating: card_data/20_template.jpg  
   creating: card_data/19_image/
  inflating: card_data/19_image/10007403114001.pdf_49.jpg  
  inflating: card_data/19_image/10007403114001.pdf_44.jpg  
  inflating: card_data/19_image/10007403114001.pdf_48.jpg  
  inflating: card_data/19_image/10007403114001.pdf_41.jpg  
  inflating: card_data/19_image/10007403114001.pdf_45.jpg  
  inflating: card_data/19_image/10007403114001.pdf_46.jpg  
  inflating: card_data/19_image/10007403114001.pdf_47.jpg  
  inflating: card_data/19_image/10007403114001.pdf_43.jpg  
  inflating: card_data/19_image/10007403114001.pdf_38.jpg  
  inflating: card_data/19_image/10007403114001.pdf_40.jpg  


In [ ]:
def args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--images', type=str, default='images', help='Path to the dir images')
    parser.add_argument('--template', type=str, default='template.jpg', help='Path to the template')
    args = parser.parse_args()
    return args

def read_image(image_file, template_file):
    img_rgb = cv.imread(image_file)
    assert img_rgb is not None, "file could not be read, check with os.path.exists()"
    image_h, image_w = img_rgb.shape[:2]
    img_gray = cv.cvtColor(img_rgb, cv.COLOR_BGR2GRAY)
    template = cv.imread(template_file, cv.IMREAD_GRAYSCALE)
    assert template is not None, "file could not be read, check with os.path.exists()"
    w, h = template.shape[::-1]
    res = cv.matchTemplate(img_gray,template,cv.TM_CCOEFF_NORMED)
    threshold = 0.35
    loc1 = np.where(res >= threshold)
    # print(c)
    listPt = []
    for pt in zip(*loc1[::-1]):
        if(len(listPt) == 0):
            cv.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (255,0,0), 3)
            half = cv.resize(img_rgb, (0, 0), fx = 0.2, fy = 0.2)
            # cv.imshow("result", half)
            # cv.waitKey(1)
            listPt.append(pt)
        else:
            check = False
            for mPt in listPt:
                if(abs(mPt[0] - pt[0]) < 400 and abs(mPt[1] - pt[1]) < 400):
                    check = True
                    break
            if(check == False):
                cv.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (255,0,0), 3)
                half = cv.resize(img_rgb, (0, 0), fx = 0.2, fy = 0.2)
                # cv.imshow("result", half)
                # cv.waitKey(1)
                listPt.append(pt)
    return listPt, w, h, image_w, image_h

def create_yolo_annotation(label_dir, image_file, list_pt, w, h, image_w, image_h):
    data = []
    namefile = image_file.split(".jpg")[0]
    print(label_dir + namefile + ".txt")
    for pt in list_pt:
        xcenter = (pt[0] + w/2)/image_w
        ycenter = (pt[1] + h/2)/image_h
        w_ = w/image_w
        h_ = h/image_h
        data.append(f'{0} {xcenter:.3f} {ycenter:.3f} {w_:.3f} {h_:.3f}')
        # To save the formatted XML to a file
        with open(label_dir + namefile + ".txt", "w+") as f:
            f.write('\n'.join(data))

for i in [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]:
    for image_file in os.listdir("card_data/"+ str(i)+ "_image/"):
        listPt, w, h, image_w, image_h = read_image("card_data/"+ str(i)+ "_image/" + image_file, "card_data/"+ str(i) + "_template.jpg")
        create_yolo_annotation("card_data/"+ str(i)+ "_labels/", image_file, listPt, w, h, image_w, image_h)

card_data/10_labels/10001430626001.pdf_46.txt
card_data/10_labels/10001430626001.pdf_47.txt
card_data/10_labels/10001430626001.pdf_44.txt
card_data/10_labels/10001430626001.pdf_42.txt
card_data/10_labels/10001430626001.pdf_43.txt
card_data/10_labels/10001430626001.pdf_40.txt
card_data/10_labels/10001430626001.pdf_41.txt
card_data/10_labels/10001430626001.pdf_35.txt
card_data/10_labels/10001430626001.pdf_48.txt
card_data/10_labels/10001430626001.pdf_45.txt
card_data/10_labels/10001430626001.pdf_49.txt
card_data/10_labels/10001430626001.pdf_39.txt
card_data/10_labels/10001430626001.pdf_38.txt
card_data/10_labels/10001430626001.pdf_37.txt
card_data/10_labels/10001430626001.pdf_36.txt
card_data/10_labels/10001430626001.pdf_27.txt
card_data/10_labels/10001430626001.pdf_26.txt
card_data/10_labels/10001430626001.pdf_28.txt
card_data/10_labels/10001430626001.pdf_33.txt
card_data/10_labels/10001430626001.pdf_24.txt
card_data/10_labels/10001430626001.pdf_34.txt
card_data/10_labels/10001430626001